In [ ]:
### LIBRARIES AND FUNCTIONS ###

In [ ]:
!pip install "numpy==1.21.6"
!pip install "pandas==1.3.5"
!pip install "praw==7.6.0"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 188 kB 4.1 MB/s 
     |████████████████████████████████| 54 kB 2.2 MB/s 


In [ ]:
import datetime
import praw
import pandas as pd
import numpy as np

In [ ]:
def column_statistics(df,column):
    print("max:", df[column].max())
    print("min:", df[column].min())
    print("mean:", round(df[column].mean(),2))

def column_statistics_str(df,column):
    print("max:", round(df[column].apply(len).max(),2))
    print("min:", round(df[column].apply(len).min(),2))
    print("mean:", round(df[column].apply(len).mean(),2))

def column_statistics_age(df,column):
    print("max:", round(df[column].max() / 3600, 2))
    print("min:", round(df[column].min() / 3600, 2))
    print("mean:", round(df[column].mean() / 3600, 2)) 

In [ ]:
def geometric_mean(a,b):
    return np.sqrt(a*b)

In [ ]:
### CONNECT ###

In [ ]:
with open('auth.txt', 'r') as f:
    auth = {}
    for line in f:
        line = line.split()
        auth[line[0]] = line[1]

In [ ]:
reddit = praw.Reddit(client_id = auth['client_id'],
                     client_secret = auth['client_secret'],
                     user_agent = "nlp_app",
                     check_for_async = False)

In [ ]:
reddit.read_only = True

In [ ]:
### GET DATA ###

In [ ]:
title = []
upvotes = []
ratio = []
post_date = []
pull_date = []
id = []
text = []

subreddit = "ask"

pull_limit = 1e4

In [ ]:
for sub in reddit.subreddit(subreddit).new(limit = pull_limit):
    if sub.is_self == True:
        title.append(sub.title)
        upvotes.append(sub.score)
        ratio.append(sub.upvote_ratio)
        post_date.append(int(sub.created_utc))
        id.append(sub.id)
        pull_date.append(int(datetime.datetime.now().timestamp()))
        text.append(sub.selftext)
        
for sub in reddit.subreddit(subreddit).hot(limit = pull_limit):
    if sub.is_self == True:
        title.append(sub.title)
        upvotes.append(sub.score)
        ratio.append(sub.upvote_ratio)
        post_date.append(int(sub.created_utc))
        id.append(sub.id)
        pull_date.append(int(datetime.datetime.now().timestamp()))
        text.append(sub.selftext)  

for sub in reddit.subreddit(subreddit).top(limit = pull_limit):
    if sub.is_self == True:
        title.append(sub.title)
        upvotes.append(sub.score)
        ratio.append(sub.upvote_ratio)
        post_date.append(int(sub.created_utc))
        id.append(sub.id)
        pull_date.append(int(datetime.datetime.now().timestamp()))
        text.append(sub.selftext)

for sub in reddit.subreddit(subreddit).rising(limit = pull_limit):
    if sub.is_self == True:
        title.append(sub.title)
        upvotes.append(sub.score)
        ratio.append(sub.upvote_ratio)
        post_date.append(int(sub.created_utc))
        id.append(sub.id)
        pull_date.append(int(datetime.datetime.now().timestamp()))
        text.append(sub.selftext) 
            
for sub in reddit.subreddit(subreddit).controversial(limit = pull_limit):
    if sub.is_self == True:
        title.append(sub.title)
        upvotes.append(sub.score)
        ratio.append(sub.upvote_ratio)
        post_date.append(int(sub.created_utc))
        id.append(sub.id)
        pull_date.append(int(datetime.datetime.now().timestamp()))
        text.append(sub.selftext) 

In [ ]:
rdf = pd.DataFrame({"Title":title,
                    "Id":id,
                    "Upvotes":upvotes, 
                    "Ratio":ratio, 
                    "Post_Date":post_date,
                    "Pull_Date":pull_date,
                    "Text":text
                    })

In [ ]:
rdf = rdf.sort_values(by = 'Pull_Date', ascending = False)
rdf = rdf.drop_duplicates(subset = ['Id'], keep = 'first')

In [ ]:
rdf.shape

(2893, 7)

In [ ]:
### TRANSFORMATIONS ###

In [ ]:
rdf['Age'] = rdf['Pull_Date'] - rdf['Post_Date']

rdf['Age_Hours'] = rdf['Age'].apply(lambda x: x / 3600)

In [ ]:
rdf.loc[(rdf['Age_Hours'] >= 72)].shape

(2511, 9)

In [ ]:
# drop rows less than 3 days old
rdf = rdf.loc[(rdf['Age_Hours'] >= 72)]

In [ ]:
rdf.shape

(2511, 9)

In [ ]:
if rdf.loc[(rdf['Ratio'] < 0)].shape[0] < 0:
    rdf['Ratio'] = rdf['Ratio'].apply(np.absolute)

if rdf.loc[(rdf['Upvotes'] < 0)].shape[0] < 0:
    rdf['Upvotes'] = rdf['Upvotes'].apply(np.absolute)

In [ ]:
rdf['Geometric_Mean'] = rdf.apply(lambda x: geometric_mean(x.Upvotes, x.Ratio), axis=1)

In [ ]:
set(rdf['Title'].apply(type)) == {str}

True

In [ ]:
if set(rdf['Text'].apply(type)) != {str}:
    rdf['Text'] = rdf['Text'].fillna('')

In [ ]:
set(rdf['Text'].apply(type)) == {str}

True

In [ ]:
rdf['Combine'] = rdf['Title'] + ' ' + rdf['Text']

In [ ]:
### INSPECT DATA ###

In [ ]:
rdf.head()

,Title,Id,Upvotes,Ratio,Post_Date,Pull_Date,Text,Age,Age_Hours,Geometric_Mean,Combine
3770,Why are trans people so easily offended?,lf948i,0,0.40,1612780288,1654469252,,41688964,11580.267778,0.000000,Why are trans people so easily offended?
3762,Why are Canadians the only here is my next att...,ntp1c1,1,0.56,1622995498,1654469252,,31473754,8742.709444,0.748331,Why are Canadians the only here is my next att...
3763,Girls don’t fart prove me wrong,nk34xe,1,0.55,1621876753,1654469252,In my 17 years of existence I have not once he...,32592499,9053.471944,0.741620,Girls don’t fart prove me wrong In my 17 years...
3764,Wondering if anyone got naked punishment in sc...,nhxrit,1,0.55,1621617381,1654469252,,32851871,9125.519722,0.741620,Wondering if anyone got naked punishment in sc...
3766,Non-binary parents what do you have your kids ...,m5mtts,1,0.56,1615821171,1654469252,,38648081,10735.578056,0.748331,Non-binary parents what do you have your kids ...


In [ ]:
column_statistics(rdf,'Upvotes')

max: 3026
min: 0
mean: 185.63


In [ ]:
column_statistics(rdf,'Ratio')

max: 1.0
min: 0.2
mean: 0.73


In [ ]:
column_statistics(rdf,'Geometric_Mean')

max: 53.33329166665039
min: 0.0
mean: 8.26


In [ ]:
column_statistics_age(rdf,'Age_Hours')

max: 27.5
min: 0.02
mean: 3.85


In [ ]:
column_statistics_str(rdf,'Text')

max: 4785
min: 0
mean: 190.83


In [ ]:
### SAVE DATAFRAME ###

In [ ]:
timestamp = datetime.datetime.now().strftime('%Y-%m-%d__%H-%M-%S')
filename = str(subreddit) + '__' + str(timestamp) + '.csv'
rdf.to_csv(filename, index= False)